Note: 

To run the bias analysis for the current process, you need a dataset that contains the IC screenings. The easiest way to do this is to run `train_model.py` after adjusting the `handling_types` parameter in the `config.yml` to include: `["is_onderzoek_hh", "is_screening_hh", "is_screening_ic"]`. Then the `BIAS_X_test.csv` and `BIAS_y_test.csv` will include everything. Since the model will also get retrained on the IC screenings, which we don't want, you should copy the two saved files to elsewhere, then revert the `config.yml` and retrain to get the original model back.

# Imports + settings

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import joblib
from collections import defaultdict
import json
import datetime as dt
from pathlib import Path

# To display BSNs fully
pd.set_option("display.max_colwidth", 1000)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
from wpi_onderzoekswaardigheid_aanvraag.project_paths import ARTIFACT_PATH, DATA_PATH, CONFIG_PATH, INFO_PATH
from wpi_onderzoekswaardigheid_aanvraag.model.manage_model_info import load_feature_list
from wpi_onderzoekswaardigheid_aanvraag.model.build_model import filter_application_handling
from wpi_onderzoekswaardigheid_aanvraag.settings.settings import WPISettings
from wpi_onderzoekswaardigheid_aanvraag.components import SocratesDienstPersoonJoin, SocratesAdresFeatures

WPISettings.set_from_yaml(CONFIG_PATH);

In [ ]:
from bias_collection.bias_analyzer import BiasAnalyzer
from fraude_preventie.datasources.dbutils import db_url_from_config

# Load data

In [ ]:
model_file = ARTIFACT_PATH / "model.pkl"
model_dict = joblib.load(model_file)
_model = model_dict["model"]

In [ ]:
transformations = _model[:-1]  # all but the last pipeline steps, hence all transformers, but not the model
model = _model[-1]  # the actual model

num_cols, cat_cols = load_feature_list()
label = "onderzoekswaardig"
X_test = pd.read_csv(DATA_PATH / "BIAS_X_test.csv")
y_test = pd.read_csv(DATA_PATH / "BIAS_y_test.csv")

# Prepare the analysis input

#### Start

In [ ]:
# Note that we need to use X_test for this rather than the transformed data, because
# to do the joins correctly we need some columns that are not in the transformed data
# anymore.
X_test_enriched = X_test.reset_index()

In [ ]:
connection_info = WPISettings.get_settings()["connections"]["basisinformatie_db"];
connection_info["options"] = ""

Get from Postgresql DB - WPI dump
- leeftijd (age)
- nationaliteit (nationality)
- geslacht (gender)
- postcode

Get from Postgresql DB - BRP dump
- geboorteland (country of birth)
- burgerlijke staat (civil status)

#### Add geslacht, nationaliteit, leeftijd

In [ ]:
sql_query = """with ref as (
    select attribuut_waarde, attribuut_waarde_omschrijving
    from wpi_hashed.socrates_ref
    where attribuut = 'NATIONALITEIT1'
)
select subjectnr, dtopvoer, dtafvoer, dtgeboortegba, geslacht, nationaliteit1, attribuut_waarde_omschrijving as nationaliteit
from wpi_hashed.socrates_persoon sp
left join ref on sp.nationaliteit1 = ref.attribuut_waarde"""

nationaliteit_df = pd.read_sql(sql_query, db_url_from_config(connection_info))

In [ ]:
X_test_enriched = SocratesDienstPersoonJoin.join_dienst_persoon(X_test_enriched, nationaliteit_df)

In [ ]:
X_test_enriched["leeftijd"] = X_test_enriched["dtaanvraag"].astype("datetime64").dt.year - X_test_enriched["dtgeboortegba_persoon"].astype("datetime64").dt.year

In [ ]:
sql_query = """select attribuut_waarde, attribuut_waarde_omschrijving
from wpi_hashed.socrates_ref
where attribuut = 'NATIONALITEIT1'
"""

nationaliteit_mapping = pd.read_sql(sql_query, db_url_from_config(connection_info)).set_index("attribuut_waarde")["attribuut_waarde_omschrijving"].to_dict()

#### Add postcode

In [ ]:
from wpi_onderzoekswaardigheid_aanvraag.preprocessing.clean import WPICleanTransformer

In [ ]:
sql_query = """select subjectnr, dtbegin, dteinde, dtopvoer, dtafvoer, postcodenum, geldig
from wpi_hashed.socrates_adres sp
"""

postcode_df = pd.read_sql(sql_query, db_url_from_config(connection_info))

In [ ]:
postcode_df = WPICleanTransformer(
    remove_invalidated_data=True,
    col_type_mapping=[
        ("dtbegin", "datetime64"),
        ("dteinde", "datetime64"),
        ("dtopvoer", "datetime64"),
        ("dtafvoer", "datetime64"),
    ],
    fix_no_end_date=["dteinde"],
).transform(postcode_df)

In [ ]:
df_tmp = SocratesAdresFeatures.join_applications_adres(X_test_enriched, postcode_df)
df_tmp = SocratesAdresFeatures.filter_adres_relevant_to_application(df_tmp)
X_test_enriched = df_tmp.sort_values("dtbegin_adres").drop_duplicates(
    "application_dienstnr", keep="last"
)

#### Add BSN from WPI data in order to join with BRP data

In [ ]:
sql_query = """select subjectnr, bsn, dtopvoer
from wpi_hashed.socrates_persoon
where bsn != 'eb763221a7e6f47e6c8f5062f8fd1ad18a95264c7366928afc8ed92e7d1917a3'
"""

bsn_df = pd.read_sql(sql_query, db_url_from_config(connection_info))

In [ ]:
# Filter BSNs on the subject numbers that we need, then remove duplicates.
relevant_bsns = bsn_df[bsn_df["subjectnr"].isin(X_test_enriched["subjectnr"].unique())].drop_duplicates()
shape_step1 = relevant_bsns.shape
relevant_bsns = relevant_bsns.sort_values("dtopvoer", ascending=True).drop_duplicates("subjectnr", keep="last")
shape_step2 = relevant_bsns.shape

if shape_step1 != shape_step2:
    print("Warning: There were people with more than 1 BSN, for them the last known BSN is used.")

In [ ]:
old_shape = X_test_enriched.shape
X_test_enriched = X_test_enriched.merge(relevant_bsns, how="left", on="subjectnr")
new_shape = X_test_enriched.shape

# Assert that the number of rows didn't change. If it did, we have subject numbers with more than 1 BSN!
assert old_shape[0] == new_shape[0]

#### Add geboorteland

In [ ]:
sql_query = """select bsn, geboorteland
from bias_analyse_wpi.brp_rapport
"""

geboorteland_df = pd.read_sql(sql_query, db_url_from_config(connection_info))

In [ ]:
geboorteland_df = geboorteland_df.drop_duplicates()

In [ ]:
old_shape = X_test_enriched.shape
X_test_enriched = X_test_enriched.merge(geboorteland_df, how="left", on="bsn")
new_shape = X_test_enriched.shape

# Assert that the number of rows didn't change. If it did, we have BSNs with more than 1 geboorteland.
assert old_shape[0] == new_shape[0]

X_test_enriched["geboorteland"] = pd.Categorical(X_test_enriched['geboorteland'])
X_test_enriched["geboorteland_code"] = X_test_enriched['geboorteland'].cat.codes

geboorteland_mapping = dict(enumerate(X_test_enriched["geboorteland"].cat.categories))

#### Add burgerlijke staat

- H = huwelijk
- P = geregistreerd partnerschap

In [ ]:
sql_query = """select bsn, soort_verbintenis, datum_sluiting, datum_ontbinding
from bias_analyse_wpi.brp_rapport
"""

burg_staat_df = pd.read_sql(sql_query, db_url_from_config(connection_info))

In [ ]:
burg_staat_df["datum_sluiting"] = pd.to_datetime(burg_staat_df["datum_sluiting"].replace(dt.date(1001, 1, 1), pd.Timestamp.min))
burg_staat_df["datum_ontbinding"] = pd.to_datetime(burg_staat_df["datum_ontbinding"].replace(dt.date(1001, 1, 1), pd.Timestamp.min))

In [ ]:
def get_civil_status_at_date(civil_status_df, bsn, date):
    df = civil_status_df[civil_status_df["bsn"] == bsn]
    df = df[df["datum_sluiting"].isna() | (df["datum_sluiting"] <= date)]
    
    if len(df) == 0:
        logger.warning(f"BSN not found in dataframe, assuming that civil status is 'single': {bsn}")
        return "single"
    
    # `datum_sluiting` is always filled in our dump for marriage/partnership (H/P).
    # So if all NaN, then there no partnership/marriage in the BRP.
    if df["datum_sluiting"].isna().mean() == 1:
        civil_status = "single"
        
    else:            
        # Check if last available partnership/marriage is still current.
        df = df.sort_values("datum_sluiting", ascending=False).drop_duplicates(subset=["bsn"], keep="first")
        
        if df["datum_ontbinding"].isna().mean() == 1:
            civil_status = "partnership_or_married"
            
        else:
            civil_status = "separated_or_divorced_or_widowed"
    
    return civil_status

In [ ]:
X_test_enriched["burgerlijke_staat"] = [get_civil_status_at_date(burg_staat_df, row["bsn"], row["dtaanvraag"]) for i, row in X_test_enriched.iterrows()]

X_test_enriched["burgerlijke_staat"] = pd.Categorical(X_test_enriched['burgerlijke_staat'])
X_test_enriched["burgerlijke_staat_code"] = X_test_enriched['burgerlijke_staat'].cat.codes

burgerlijke_staat_mapping = dict(enumerate(X_test_enriched["burgerlijke_staat"].cat.categories))

#### Prepare final dataframe

In [ ]:
X_test_enriched = X_test_enriched.rename(columns={
    "nationaliteit1_persoon": "nationaliteit_code",
    "postcodenum_adres": "postcodenum",
    "geslacht_persoon": "geslacht",
})

external_bias_columns = [
    "geslacht",
    "leeftijd",
    "nationaliteit_code",
    "postcodenum",
    "geboorteland_code",
    "burgerlijke_staat_code",
]

data_to_analyze = transformations.transform(X_test)
data_to_analyze[label] = y_test[label].replace({True: 1, False: 0})
data_to_analyze.index = X_test["application_dienstnr"]

data_to_analyze = data_to_analyze.merge(X_test_enriched.set_index("application_dienstnr")[external_bias_columns], left_index=True, right_index=True)
data_to_analyze = data_to_analyze.dropna()

data_to_analyze.head()

# Make groups

In [ ]:
features_to_check = defaultdict(list)

## Direct

#### Sex

- 0 = unknown
- 1 = male
- 2 = female

Note that we only compare male vs. female, because we don't have enough samples with unknown gender.

In [ ]:
data_to_analyze["geslacht"].value_counts()

In [ ]:
# data_to_analyze = data_to_analyze[data_to_analyze["geslacht"] != 0]
features_to_check["geslacht"] = [[1], [2]]

#### Age

In [ ]:
data_to_analyze["leeftijd"].describe()

In [ ]:
data_to_analyze["leeftijd"].hist()

In [ ]:
features_to_check["leeftijd_split1"] = [0, 100, 40, 1]
features_to_check["leeftijd_split2"] = [0, 100, 50, 1]

#### Nationality

In [ ]:
with open("west-nonwest-nationalities.json", 'r') as j:
    west_nonwest_nationalities = json.loads(j.read())

In [ ]:
for code, nationality in zip(data_to_analyze["nationaliteit_code"].value_counts().iteritems(), data_to_analyze["nationaliteit_code"].map(nationaliteit_mapping).value_counts().iteritems()):
    print(f"Count: {nationality[1]:<5} Code: {int(code[0]):<5} {nationality[0]:<20}")

In [ ]:
flipped_mapping = {v: k for k,v in nationaliteit_mapping.items()}
west_codes = [code for country, code in flipped_mapping.items() if country in west_nonwest_nationalities["west"]]
nonwest_codes = [code for country, code in flipped_mapping.items() if country in west_nonwest_nationalities["nonwest"]]

In [ ]:
data_to_analyze_no_unknown_nationality = data_to_analyze[data_to_analyze["nationaliteit_code"] != 0]
# Check that all nationality codes got assigned to west/nonwest except 0 = unknown.
assert (data_to_analyze_no_unknown_nationality["nationaliteit_code"].isin(west_codes) | data_to_analyze_no_unknown_nationality["nationaliteit_code"].isin(nonwest_codes)).all()

In [ ]:
features_to_check["nationaliteit_code_split1"] = [west_codes, nonwest_codes]  # West vs. non-west
features_to_check["nationaliteit_code_split2"] = [
    [1], 
    [n for n in data_to_analyze["nationaliteit_code"] if n not in [0, 1]]  # Dutch vs. non-Dutch
]

#### Country of birth

In [ ]:
with open("west-nonwest-countries.json", 'r') as j:
    west_nonwest_countries = json.loads(j.read())

In [ ]:
for code, country in zip(data_to_analyze["geboorteland_code"].value_counts().iteritems(), data_to_analyze["geboorteland_code"].map(geboorteland_mapping).value_counts().iteritems()):
    print(f"Count: {country[1]:<5} Code: {int(code[0]):<5} {country[0]:<20}")

In [ ]:
flipped_mapping = {v: k for k,v in geboorteland_mapping.items()}
west_codes = [code for country, code in flipped_mapping.items() if country in west_nonwest_countries["west"]]
nonwest_codes = [code for country, code in flipped_mapping.items() if country in west_nonwest_countries["nonwest"]]

In [ ]:
# Check that all country codes got assigned to west/nonwest.
assert (data_to_analyze["geboorteland_code"].isin(west_codes) | data_to_analyze["geboorteland_code"].isin(nonwest_codes)).all()

In [ ]:
features_to_check["geboorteland_code_split1"] = [west_codes, nonwest_codes]  # West vs. non-west
features_to_check["geboorteland_code_split2"] = [
    [39], 
    [n for n in data_to_analyze["geboorteland_code"] if n != 39]  # Dutch vs. non-Dutch
]

#### Civil status

In [ ]:
for code, burg_staat in zip(data_to_analyze["burgerlijke_staat_code"].value_counts().iteritems(), data_to_analyze["burgerlijke_staat_code"].map(burgerlijke_staat_mapping).value_counts().iteritems()):
    print(f"Count: {burg_staat[1]:<5} Code: {int(code[0]):<5} {burg_staat[0]:<20}")

In [ ]:
priv = [2]  # single
unpriv = [0, 1]
features_to_check["burgerlijke_staat_code"] = [priv, unpriv]

## Indirect

#### Feature: deelnames_started_percentage_last_year

In [ ]:
data_to_analyze["deelnames_started_percentage_last_year"].hist()

In [ ]:
data_to_analyze["deelnames_started_percentage_last_year"].value_counts()

In [ ]:
data_to_analyze["deelnames_started_percentage_last_year_equals_zero"] = (data_to_analyze["deelnames_started_percentage_last_year"] == 0)*1
data_to_analyze["deelnames_started_percentage_last_year_equals_one"] = (data_to_analyze["deelnames_started_percentage_last_year"] == 1)*1

# This means: People who started nothing last year (incl. those who weren't in the system last year) vs. people who started something or everything.
features_to_check["deelnames_started_percentage_last_year_equals_zero"] = [
    [0], [1]
]
# This means: People who started everything last year vs. those who didn't start everything or who weren't in the system last year.
features_to_check["deelnames_started_percentage_last_year_equals_one"] = [
    [0], [1]
]

#### Feature: at_least_one_address_in_amsterdam

In [ ]:
data_to_analyze["at_least_one_address_in_amsterdam"].value_counts()

In [ ]:
features_to_check["at_least_one_address_in_amsterdam"] = [
    [0], [1]
]

#### Feature: active_address_count

In [ ]:
data_to_analyze["active_address_count"].value_counts()

In [ ]:
features_to_check["active_address_count"] = [
    [1], [2, 3]
]

#### Feature: days_since_last_relocation

In [ ]:
data_to_analyze["days_since_last_relocation"].describe()

In [ ]:
data_to_analyze["days_since_last_relocation"].hist()

In [ ]:
data_to_analyze["days_since_last_relocation"].unique()

In [ ]:
split_value = 365
features_to_check["days_since_last_relocation"] = [
    [n for n in data_to_analyze["days_since_last_relocation"].unique() if n > split_value],  # Same address for a long time
    [n for n in data_to_analyze["days_since_last_relocation"].unique() if n <= split_value]  # Moved in the past year
]

#### Feature: days_since_last_dienst_end

In [ ]:
data_to_analyze["days_since_last_dienst_end"].hist()

In [ ]:
features_to_check["days_since_last_dienst_end_split1"] = [
    [99999],  # No dienst last year
    [n for n in data_to_analyze["days_since_last_dienst_end"].unique() if n != 99999]  # Had a dienst last year
]

split_value = 60
features_to_check["days_since_last_dienst_end_split2"] = [
    [n for n in data_to_analyze["days_since_last_dienst_end"].unique() if n > split_value],  # Dienst longer than 60 days ago
    [n for n in data_to_analyze["days_since_last_dienst_end"].unique() if n <= split_value]  # Dienst within last 60 days
]

#### Feature: has_medebewoner

In [ ]:
data_to_analyze["has_medebewoner"].value_counts()

In [ ]:
features_to_check["has_medebewoner"] = [
    [0], [1]
]

#### Feature: avg_percentage_maatregel

In [ ]:
data_to_analyze["avg_percentage_maatregel"].value_counts()

I think we have too few samples to say anything meaningful here.

#### Feature: total_vermogen

In [ ]:
data_to_analyze["total_vermogen"].describe()

In [ ]:
data_to_analyze["total_vermogen"].hist(bins=300, figsize=(15,5))

In [ ]:
split_value = 0
features_to_check["total_vermogen_split1"] = [ 
    [n for n in data_to_analyze["total_vermogen"].unique() if n >= split_value],  # Greater than or equal to zero wealth
    [n for n in data_to_analyze["total_vermogen"].unique() if n < split_value]    # Negative wealth
]

split_value = 0
features_to_check["total_vermogen_split2"] = [ 
    [n for n in data_to_analyze["total_vermogen"].unique() if n > split_value],  # Positive wealth
    [n for n in data_to_analyze["total_vermogen"].unique() if n < split_value]    # Negative wealth
]

#### Feature: afspraken_no_show_count_last_year

In [ ]:
data_to_analyze["afspraken_no_show_count_last_year"].value_counts()

In [ ]:
features_to_check["afspraken_no_show_count_last_year"] = [
    [0], [1, 2, 3]
]

#### Feature: has_partner

In [ ]:
data_to_analyze["has_partner"].value_counts()

In [ ]:
features_to_check["has_partner"] = [
    [0], [1]
]

#### Feature: sum_inkomen_bruto_was_mean_imputed

In [ ]:
data_to_analyze["sum_inkomen_bruto_was_mean_imputed"].value_counts()

In [ ]:
features_to_check["sum_inkomen_bruto_was_mean_imputed"] = [
    [0], [1]
]

#### Feature: applied_for_same_product_last_year

In [ ]:
data_to_analyze["applied_for_same_product_last_year"].value_counts()

In [ ]:
features_to_check["applied_for_same_product_last_year"] = [
    [0], [1]
]

#### Feature: received_same_product_last_year

In [ ]:
data_to_analyze["received_same_product_last_year"].value_counts()

In [ ]:
features_to_check["received_same_product_last_year"] = [
    [0], [1]
]

#### Feature: afspraken_no_contact_count_last_year

In [ ]:
data_to_analyze["afspraken_no_contact_count_last_year"].value_counts()

I think we have too few samples to say anything meaningful here.

#### Feature: sum_inkomen_bruto_value

In [ ]:
data_to_analyze["sum_inkomen_bruto_value"].describe()

In [ ]:
features_to_check["sum_inkomen_bruto_value"] = [ 
    [0],  # No income
    [n for n in data_to_analyze["sum_inkomen_bruto_value"].unique() if n > 0]    # Has non-zero income
]

# Do analysis

In [ ]:
external_variables = external_bias_columns + [
    "deelnames_started_percentage_last_year_equals_zero", 
    "deelnames_started_percentage_last_year_equals_one"
]

BiasAnalyzer(
    [
        "false_positive_rate_difference",
        "false_positive_rate_ratio",
        "false_positive_group_size_difference",
        "false_positive_group_size_ratio",
    ]
).analyze_features(
    data_to_analyze=data_to_analyze,
    model=_model.named_steps["clf"].best_estimator_,
    sensitive_features=features_to_check,
    outpath=Path(INFO_PATH) / "bias_report",
#     thresholds=np.arange(0.4, 0.6, 0.01),
    label_column_name=label,
    external_variables=external_variables,
#     print_metric_explanations=True,
)



In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(BiasAnalyzer.log_info_by_metric)

In [ ]:
# Don't run the stuff below
assert False

- False discovery rate difference is positive (0.011), meaning that the female group is (barely) disadvantaged.
- False positive rate difference is negative (-0.089), meaning that the male group is disadvantaged.
- False positive/group size difference is negative (-0.054), meaning that the male group is disadvantaged.

#### False discovery rate difference interpretation
FDR male = 0.1
Out of all the males we investigate, 10% are actually innocent.

FDR female = 0.1 + 0.011 = 0.111
Out of all the females we investigate, 11.1% are actually innocent.

If we investigate a woman, she is 1.1 percentage points more likely to be innocent than a man we investigate.


#### False positive rate difference interpretation
FPR male = 0.1
If you are an innocent male, then you have a 10% chance of being investigated anyway.

FPR female = 0.1 - 0.089 = 0.011
If you are an innocent female, then you have a 1.1% chance of being investigated anyway.

The chance of being investigated as an innocent male is 8.9 percentage points higher than as an innocent female.


#### False positive/group size difference interpretation
FP/GS male = 0.089
A random man has a 8.9% chance to be wrongly investigated.

FP/GS female = 0.035
A random woman has a 3.5% chance to be wrongly investigated.

The chance of being wrongly investigated for a random man is 5.4 percentage points higher than for a random woman.

In [ ]:
analysis_df[["geslacht", "onderzoekswaardig"]].value_counts()

In [ ]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Test
logger = logging.getLogger()

In [ ]:
analysis_df.groupby("geslacht")["onderzoekswaardig"].mean()

#### False positive/group size
P(Y_hat=1, Y=0|group)

In [ ]:
analysis_df_with_preds = analysis_df.copy()
analysis_df_with_preds["y_pred"] = model.predict_proba(analysis_df.drop(["geslacht", "onderzoekswaardig"], axis=1))[:, 1]

In [ ]:
male_df = analysis_df_with_preds[analysis_df_with_preds["geslacht"]==1]
male_fp_group_size = ((male_df["y_pred"] > 0.5) & (male_df["onderzoekswaardig"] == 0)).mean()

In [ ]:
female_df = analysis_df_with_preds[analysis_df_with_preds["geslacht"]==2]
female_fp_group_size = ((female_df["y_pred"] > 0.5) & (female_df["onderzoekswaardig"] == 0)).mean()

In [ ]:
print(f"Male: {male_fp_group_size:.3f}")
print(f"Female: {female_fp_group_size:.3f}")
print(f"False positive/group size difference: {female_fp_group_size - male_fp_group_size:.3f}")